<a href="https://colab.research.google.com/github/Lailson001/E-comerce-modelo/blob/main/An%C3%A1lise%20E-comerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

import numpy as np
import matplotlib




# Nova seção

🚀 Limpando e Validando Dados com Pandas!

In [31]:
# Lendo o arquivo CSV e carregando os dados em um DataFrame
df = pd.read_csv("/content/retail_store_sales.csv")

# Exibindo as primeiras linhas do DataFrame para verificar a estrutura
df.head()

# Exibindo informações sobre o DataFrame, como tipos
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Transaction ID      12575 non-null  object 
 1   Customer ID         12575 non-null  object 
 2   Category            12575 non-null  object 
 3   Item                12575 non-null  object 
 4   Price Per Unit      12575 non-null  float64
 5   Quantity            12575 non-null  float64
 6   Total Spent         12575 non-null  float64
 7   Payment Method      12575 non-null  object 
 8   Location            12575 non-null  object 
 9   Transaction Date    12575 non-null  object 
 10  Discount Applied    12575 non-null  object 
 11  Calculated Total    11971 non-null  float64
 12  Inconsistence Type  12575 non-null  object 
dtypes: float64(4), object(9)
memory usage: 1.2+ MB
None


Ao indentificar os valores ausentes, usamos cálculos para preenchê-los com dados coerentes. Por exemplo, completamos Total Spent multiplicando Quantity por Price Per Unit. Isso ajuda a manter a consistência das informações.

In [32]:
# Preenchendo valores ausentes na coluna 'Total Spent' multiplicando 'Quantity' por 'Price Per Unit'
df['Total Spent'] = df['Total Spent'].fillna(df['Quantity'] * df['Price Per Unit'])

# Preenchendo valores ausentes na coluna 'Price Per Unit' dividindo 'Total Spent' por 'Quantity'
df['Price Per Unit'] = df['Price Per Unit'].fillna(df['Total Spent'] / df['Quantity'])

# Exibindo a contagem de valores nulos por coluna após o preenchimento
print(df.isnull().sum())

# Criando uma nova coluna 'Calculated Total' para validar os cálculos
df['Calculated Total'] = df['Quantity'] * df['Price Per Unit']

Transaction ID          0
Customer ID             0
Category                0
Item                    0
Price Per Unit          0
Quantity                0
Total Spent             0
Payment Method          0
Location                0
Transaction Date        0
Discount Applied        0
Calculated Total      604
Inconsistence Type      0
dtype: int64


In [33]:
# Identificando transações inconsistentes (onde Total Spent não bate com Calculated Total dentro da margem de erro)
inconsistentes = df[~np.isclose(df['Total Spent'], df['Calculated Total'], atol=0.01)]

# Exibindo a quantidade de transações inconsistentes
print(f"Existem {len(inconsistentes)} transações inconsistentes.")

# Criando uma nova coluna para armazenar o tipo de inconsistência
df['Inconsistence Type'] = 'Consistent'


Existem 0 transações inconsistentes.


In [34]:
# Identificando os diferentes tipos de inconsistência

df.loc[df['Total Spent'].isna(), 'Inconsistence Type'] = 'missing Total Spent'  # Se 'Total Spent' estiver ausente
df.loc[df['Price Per Unit'].isna(), 'Inconsistence Type'] = 'missing Price Per Unit'  # Se 'Price Per Unit' estiver ausente
df.loc[df['Quantity'].isna(), 'Inconsistence Type'] = 'missing Quantity'  # Se 'Quantity' estiver ausente

# Identificando casos onde o cálculo não bate dentro da margem de erro
df.loc[~np.isclose(df['Total Spent'], df['Calculated Total'], atol=0.01),
       'Inconsistence Type'] = 'Incorrect Calculation'

# Filtrando apenas os registros inconsistentes para exibição
inconsistentes = df[df['Inconsistence Type'] != 'Consistent']
print(inconsistentes)


Empty DataFrame
Columns: [Transaction ID, Customer ID, Category, Item, Price Per Unit, Quantity, Total Spent, Payment Method, Location, Transaction Date, Discount Applied, Calculated Total, Inconsistence Type]
Index: []


In [35]:
# Substituindo valores nulos por 0 em colunas numéricas para evitar problemas de cálculo
df.loc[df['Total Spent'].isna(), 'Total Spent'] = 0
df.loc[df['Price Per Unit'].isna(), 'Price Per Unit'] = 0
df.loc[df['Quantity'].isna(), 'Quantity'] = 0

# Substituindo valores nulos por 0 nas colunas categóricas 'Item' e 'Discount Applied'
df[['Item', 'Discount Applied']] = df[['Item', 'Discount Applied']].fillna(0)

# Exibindo novamente a contagem de valores nulos para verificar se foram tratados
print(df[['Price Per Unit', 'Quantity', 'Total Spent', 'Item', 'Discount Applied']].isnull().sum())

Price Per Unit      0
Quantity            0
Total Spent         0
Item                0
Discount Applied    0
dtype: int64
